Used for filtering only tn datasets

In [1]:
from dotenv import load_dotenv
import os
from pathlib import Path
import pandas as pd
from datetime import datetime, timedelta
from tqdm.auto import tqdm

load_dotenv()
data_fol = Path(os.getenv("DATA_FOL"))
v1 = data_fol / 'v1'
tn_v1 = data_fol / 'tn_v1'


tn_districts = {"Ariyalur", "Chengalpattu", "Chennai", "Coimbatore", "Cuddalore", "Dharmapuri", "Dindigul", "Erode", "Kallakurichi", "Kancheepuram", "Kanyakumari", "Karur", "Krishnagiri", "Madurai", "Mayiladuthurai", "Nagapattinam", "Namakkal", "Nilgiris", "Perambalur", "Pudukkottai", "Ramanathapuram", "Ranipet", "Salem", "Sivaganga", "Tenkasi", "Thanjavur", "Theni", "Thoothukudi", "Tiruchirappalli", "Tirunelveli", "Tirupathur", "Tiruppur", "Tiruvallur", "Tiruvannamalai", "Tiruvarur", "Vellore", "Viluppuram", "Virudhunagar",}

In [2]:
local = []
got_district = set()

def week_day(some_date):
    return some_date - timedelta(days=some_date.weekday())

def tn_values_only(row):
    if row['District Name'] not in tn_districts:
        return

    got_district.add(row['District Name'])
    local.append([week_day(row['Price Date']), float(row['Min Price (Rs./Quintal)']), float(row['Max Price (Rs./Quintal)']), float(row['Modal Price (Rs./Quintal)'])])



In [4]:
for file in tqdm(os.listdir(v1)):
    df = pd.read_feather(v1/file)
    df['Price Date'] = pd.to_datetime(df['Price Date'])
    local = []
    df.apply(tn_values_only, axis=1)
    new_df = pd.DataFrame(local, columns=['date', 'min', 'max', 'modal'])

    filter_df = new_df.groupby(by=['date']).mean().round(2)
    filter_df = filter_df.sort_values('date')
    filter_df = filter_df.reset_index()

    filter_df.to_feather(tn_v1/file)

  0%|          | 0/325 [00:00<?, ?it/s]